In [32]:
import tkinter as tk
from tkinter import ttk
from openpyxl import Workbook
from openpyxl import load_workbook
from datetime import date

class Onibus:
    def __init__(self, capacidade):
        self.capacidade = capacidade
        self.lugares = [0] * capacidade

    def reservar_lugar(self,num_lugar,nome,cpf,dia):
        if num_lugar < 1 or num_lugar > self.capacidade:
            return "Lugar inválido!"
        
        if self.lugares[num_lugar - 1] == 0:
            self.lugares[num_lugar - 1] = 1
            self.salvar_reserva(num_lugar,nome,cpf,dia)

            return f"Lugar {num_lugar} reservado com sucesso."
        else:
            return f"Lugar {num_lugar} indisponível"
        
    def cancelar_reserva(self,num_lugar):
        if num_lugar < 1 or num_lugar > self.capacidade:
            resultado_label["text"] = "Lugar inválido!"
            return "Lugar inválido!"
        
        if self.lugares[num_lugar - 1] == 1:
            self.lugares[num_lugar - 1] = 0
            self.excluir_reserva(num_lugar)  
            
            resultado_label["text"] = f"Lugar {num_lugar} cancelada com sucesso."      

            return f"Lugar {num_lugar} cancelada com sucesso."
        else:
            resultado_label["text"] = f"Lugar {num_lugar} não esta resevado" 
            return f"Lugar {num_lugar} não esta resevado"
        
    def excluir_reserva(self,num_lugar):
        try:
            workbook = load_workbook("Dados.xlsx")
            sheet = workbook['Reservas']

        except FileNotFoundError:
            print("Arquivo não encontrado")
            return
        
        data = entry_data.get()
        
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
            if row[0].value == num_lugar and row[3].value == data:
                sheet.delete_rows(row[0].row)
                workbook.save("Dados.xlsx")

                break

        onibus.carregar_reservas(data)        
        
    def gerar_mapa(self,data):
        mapa = ""
        for i in range(0, self.capacidade,2):
            status_esquerda = " "
            status_direita = " "

            lugar_esquerda = i + 1
            lugar_direita = i + 2

            status_esquerda = "X" if self.lugares[i] == 1 else " "
            status_direita = "X" if lugar_direita <= self.capacidade and self.lugares[i+1] == 1 else " "

            mapa += f"Lugar {lugar_esquerda}: [{status_esquerda}]   Lugar {lugar_direita}: [{status_direita}]\n"

        return mapa

    
    def salvar_reserva(self,num_lugar,nome,cpf,dia):
        linha = [num_lugar,nome,cpf,dia]

        try:
            workbook = load_workbook("Dados.xlsx")
            sheet = workbook['Reservas']

        except FileNotFoundError:
            print("Arquivo não encontrado")
        
        sheet.append(linha)
        workbook.save("Dados.xlsx")  

    def carregar_reservas(self,data):
        try:
            workbook = load_workbook("Dados.xlsx")
            sheet = workbook['Reservas']

        except FileNotFoundError:
            return
        
        self.lugares = [0] * self.capacidade

        for i, linha in enumerate(sheet.iter_rows(values_only=True), start=1):
            if i == 1:
                continue

            num_lugar = linha[0]

            if num_lugar and 1 <= num_lugar <= self.capacidade:
                if linha[3] == data:
                    self.lugares[num_lugar - 1] = 1
        
        mapa_text.delete("1.0",tk.END)
        mapa_text.insert(tk.END,onibus.gerar_mapa(data))

def reservar_lugar():
    num_lugar = int(lugar_entry.get())
    nome = nome_entry.get()
    cpf = cpf_entry.get()
    dia = dia_entry.get()

    resultado = onibus.reservar_lugar(num_lugar,nome,cpf,dia)

    resultado_label["text"] = resultado

    mapa_text.delete("1.0",tk.END)

    onibus.carregar_reservas(entry_data.get())

def ver_reservas():
    try:
        workbook = load_workbook("Dados.xlsx")
        sheet = workbook['Reservas']

    except FileNotFoundError:
        print("Arquivo não encontrado")

    janela_reserva = tk.Toplevel(janela)
    janela_reserva.title("Reservas")
    janela_reserva.geometry(f"1300x500")

    largura_Tela = janela_reserva.winfo_screenwidth()
    altura_Tela = janela_reserva.winfo_screenheight()

    pos_x = int(largura_Tela / 2 - 400 / 2)
    pos_y = int(altura_Tela / 2 - 750 / 2)

    janela_reserva.geometry(f"1300x500+{pos_x}+{pos_y}")

    frame_filtros = tk.Frame(janela_reserva)
    frame_filtros.pack(pady=10)

    rotulos_filtro = ["Lugar","Nome","CPF","Data"]

    campos_filtros = []

    for rotulo in rotulos_filtro:
        rotulo_label = tk.Label(frame_filtros,
                                text=rotulo+": ",
                                font=("Arial",14))
        rotulo_label.pack(side=tk.LEFT,padx=10)
        rotulo_entry = tk.Entry(frame_filtros,
                                font=("Arial",14))
        rotulo_entry.pack(side=tk.LEFT)

        campos_filtros.append(rotulo_entry)

    botao_filtrar = tk.Button(frame_filtros,
                            text="Filtrar",
                            font=("Arial",14),
                            command=lambda: filtrar_reservas(sheet,rotulos_filtro,campos_filtros,treeview))
    botao_filtrar.pack(side=tk.LEFT, padx=10)

    treeview = ttk.Treeview(janela_reserva)
    treeview["columns"] = ("Lugar","Nome","CPF","Data")
    treeview.heading("Lugar",text="Lugar",anchor=tk.CENTER)
    treeview.heading("Nome",text="Nome",anchor=tk.CENTER)
    treeview.heading("CPF",text="CPF",anchor=tk.CENTER)
    treeview.heading("Data",text="Data",anchor=tk.CENTER)
    treeview.column("#0",width=0,stretch=tk.NO)

    style = ttk.Style()
    style.configure("Treeview",font=("Arial",14))
    style.configure("Treeview.Heading",font=("Arial",12,"bold"))

    treeview.column("Lugar",width=100,anchor=tk.CENTER)
    treeview.column("Nome",width=200,anchor=tk.W)
    treeview.column("CPF",width=150,anchor=tk.CENTER)
    treeview.column("Data",width=100,anchor=tk.CENTER)

    for linha in sheet.iter_rows(min_row=2,values_only=True):
        treeview.insert("",tk.END,values=linha)

    treeview.pack(pady=10,padx=10,fill=tk.BOTH,expand=True)

    scroollbar = tk.Scrollbar(janela_reserva,orient=tk.VERTICAL, command=treeview.yview)
    scroollbar.pack(side=tk.RIGHT, fill=tk.Y)

    treeview.configure(yscrollcommand=scroollbar.set)
    
def filtrar_reservas(sheet,rotulos_filtro,campos_filtros,treeview):
    treeview.delete(*treeview.get_children())

    for linha in sheet.iter_rows(min_row=2,values_only=True):
        for rotulo, campo in zip(rotulos_filtro,campos_filtros):
            if campo.get():
                if str(linha[rotulos_filtro.index(rotulo)]).lower() != campo.get().lower():
                    break
                else:
                    continue
        else:
            treeview.insert("",tk.END,values=linha)

janela = tk.Tk()
janela.title("Reserva de Passagens")
janela.geometry("400x750")
janela.configure(bg="#FFFFFF")

largura_Tela = janela.winfo_screenwidth()
altura_Tela = janela.winfo_screenheight()

pos_x = int(largura_Tela / 2 - 400 / 2)
pos_y = int(altura_Tela / 2 - 750 / 2)

janela.geometry(f"400x750+{pos_x}+{pos_y}")

onibus = Onibus(20)

titulo_label = tk.Label(janela,
                        text="Reserva de Passagens",
                        font=("Arial",16),
                        bg="#FFFFFF")
titulo_label.pack(pady=10)

formulario_frame = tk.Frame(janela,bg="#ffffff")
formulario_frame.pack(pady=10)

lugar_label = tk.Label(formulario_frame,
                        text="Número do lugar: ",
                        font=("Arial",16),
                        bg="#FFFFFF")
lugar_label.grid(row=0,column=0,sticky="e")
lugar_entry = tk.Entry(formulario_frame,font=("Arial",16))
lugar_entry.grid(row=0,column=1,sticky='e')

nome_label = tk.Label(formulario_frame,
                        text="Nome: ",
                        font=("Arial",16),
                        bg="#FFFFFF")
nome_label.grid(row=1,column=0,sticky="e")
nome_entry = tk.Entry(formulario_frame,font=("Arial",16))
nome_entry.grid(row=1,column=1,sticky='e')

nome_label = tk.Label(formulario_frame,
                        text="Nome: ",
                        font=("Arial",16),
                        bg="#FFFFFF")
nome_label.grid(row=1,column=0,sticky="e")
nome_entry = tk.Entry(formulario_frame,font=("Arial",16))
nome_entry.grid(row=1,column=1,sticky='e')

cpf_label = tk.Label(formulario_frame,
                        text="CPF: ",
                        font=("Arial",16),
                        bg="#FFFFFF")
cpf_label.grid(row=2,column=0,sticky="e")
cpf_entry = tk.Entry(formulario_frame,font=("Arial",16))
cpf_entry.grid(row=2,column=1,sticky='e')

dia_label = tk.Label(formulario_frame,
                        text="Dia: ",
                        font=("Arial",16),
                        bg="#FFFFFF")
dia_label.grid(row=3,column=0,sticky="e")
dia_entry = tk.Entry(formulario_frame,font=("Arial",16))
dia_entry.grid(row=3,column=1,sticky='e')

reserva_button = tk.Button(janela,
                            text="Reservar",
                            font=("Arial",16),
                            command=reservar_lugar)
reserva_button.pack(side='top',fill='both',padx=10,pady=2)

cancelar_button = tk.Button(janela,
                            text="Cancelar Reservar",
                            font=("Arial",16),
                            command=lambda: onibus.cancelar_reserva(int(lugar_entry.get())))
cancelar_button.pack(side='top',fill='both',padx=10,pady=2)

ver_reserva_button = tk.Button(janela,
                            text="Ver Reservas",
                            font=("Arial",16),
                            command=ver_reservas)
ver_reserva_button.pack(side='top',fill='both',padx=10,pady=2)

resultado_label = tk.Label(janela,
                            text="",
                            font=("Arial",16),
                            bg="#ffffff")
resultado_label.pack()

mapa_text = tk.Text(janela,
                    width=50,
                    height=10,
                    bg="#ffffff",
                    font=("Arial",18))
mapa_text.pack()

data_atual = date.today().strftime("%d/%m/%Y")

data_frame = tk.Frame(janela,bg="#ffffff")
data_frame.pack(pady=10)

data_label = tk.Label(data_frame,
                        text="Data: ",
                        font=("Arial",24),
                        bg="#ffffff")
data_label.grid(row=0,column=0,sticky='e')

entry_data = tk.Entry(data_frame,font=("Arial",24))
entry_data.grid(row=0,column=1,sticky='e')
entry_data.insert(tk.END,data_atual)

trazer_reserva_button = tk.Button(data_frame,
                            text="Filtrar Dia: ",
                            font=("Arial",24),
                            command=lambda: onibus.carregar_reservas(entry_data.get()))
trazer_reserva_button.grid(row=1,column=0,columnspan=2,sticky='nsew')


janela.mainloop()